In [ ]:
import pickle
import pandas as pd
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

Note: You will need to create a new project in the Google Cloud Project, you will need an api key and a OAuth 2.0 Client ID. Download the json and rename it credentials_youtube.json

In [ ]:
################## ADD API KEY ####################
api_key = "API_KEY_HERE"

ClientSecret = "credentials_youtube.json"
Scopes = ["https://www.googleapis.com/auth/youtube"]

flow = InstalledAppFlow.from_client_secrets_file(ClientSecret, Scopes)
credentials = flow.run_console()

## Saving channel data:

In [ ]:
def requestSubs(nextpageToken = None):
    request = youtube.subscriptions().list(part="snippet", mine=True, maxResults=50, fields = ('items(snippet(title,resourceId(channelId),thumbnails(high))),nextPageToken'),pageToken = nextpageToken)
    return request.execute()

def get_Upload_id(channelid):
    videos_of_subs = youtube.channels().list(part='contentDetails',id=str(channelid))
    details = videos_of_subs.execute()
    
    try:
        return details['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    except:
        return None

def pageiterator(function):
    req = function()
    all_pages = req['items']

    while True:
        if 'nextPageToken' in req.keys():
            req = requestSubs(nextpageToken = req['nextPageToken'])
            all_pages = all_pages + req['items']
        else:
            break
    return all_pages

def dict_Collapser(dict_of_dicts):
    new_dict = {}

    for item in dict_of_dicts:
        name = item['snippet']['title']
    
        df = pd.json_normalize(item['snippet'], sep='_')

        values = df.to_dict(orient='records')[0]
        new_dict[name] = values
        
    channel_data_frame = pd.DataFrame(new_dict).T
    del channel_data_frame['title']
    return channel_data_frame


youtube = build('youtube','v3',developerKey=api_key,credentials=credentials)
all_channels = pageiterator(requestSubs)
channel_data = dict_Collapser(all_channels)

channel_data['upload_playlistId']  = channel_data['resourceId_channelId'].apply(get_Upload_id)

pickle.dump(channel_data, open('channel_data_large.pkl','wb'))

In [ ]:
pickle.load(open('channel_data_large.pkl','rb'))

I have a list of channels I watch during the week and a longer list I allow myself to watch at weekends, so I create two files and normal and a large.

## Add together - to make weekend list

In [ ]:
large = pickle.load(open('channel_data_large.pkl','rb'))
normal = pickle.load(open('channel_data.pkl','rb'))

In [ ]:
large_and_normal = pd.concat([normal, large]).drop_duplicates() # Merges normal list with large list

In [ ]:
large_and_normal

In [ ]:
pickle.dump(large_and_normal, open('channel_data_large.pkl','wb'))

In [ ]:
pickle.load(open('channel_data_large.pkl','rb'))